In [1]:
class Step:
    def __init__(self, func):
        self.func = func
        self.name = func.__name__

    def __call__(self, data):
        print(f"[CALL] {self.name}({data})")
        return self.func(data)

    def __or__(self, other):
        print(f"[OR] {self.name} | {other.name}")
        return Step(lambda data: other(self(data)))

    def __ror__(self, other):
        print(f"[ROR] {other} | {self.name}")
        return self(other)

# 예시 함수들
@Step
def double(x):
    return x * 2

@Step
def increment(x):
    return x + 1

@Step
def to_str(x):
    return f"결과: {x}"

In [2]:
# 파이프라인 사용 예시
result = 3 | double | increment | to_str
print(result)  # 출력: 결과: 7

[ROR] 3 | double
[CALL] double(3)
[ROR] 6 | increment
[CALL] increment(6)
[ROR] 7 | to_str
[CALL] to_str(7)
결과: 7


In [3]:
# 파이프라인 구성 시 __or__ 호출 로그 확인
pipeline = double | increment
# 파이프라인 실행 시 __ror__, __call__ 호출 로그 확인
result = 3 | pipeline | to_str
print(result)

[OR] double | increment
[ROR] 3 | <lambda>
[CALL] <lambda>(3)
[CALL] double(3)
[CALL] increment(6)
[ROR] 7 | to_str
[CALL] to_str(7)
결과: 7


In [4]:
class Step2:
    def __init__(self, func, name=None):
        self.steps = [(func, name or func.__name__)]

    def __or__(self, other):
        combined = Step2(lambda x: x)  # identity
        combined.steps = self.steps + other.steps
        return combined

    def __ror__(self, other):
        data = other
        for func, name in self.steps:
            print(f"[CALL] {name}({data})")
            data = func(data)
        return data

    def __call__(self, data):
        return self.__ror__(data)

# 예시 함수들
@Step2
def double2(x):
    return x * 2

@Step2
def increment2(x):
    return x + 1

@Step2
def to_str2(x):
    return f"결과: {x}"

In [5]:
# 파이프라인 사용 예시
result2 = 3 | double2 | increment2 | to_str2
print(result2)  # 출력: 결과: 7

[CALL] double2(3)
[CALL] increment2(6)
[CALL] to_str2(7)
결과: 7


In [6]:
# 파이프라인 구성 시 __or__ 호출 로그 확인
pipeline2 = double2 | increment2
# 파이프라인 실행 시 __ror__, __call__ 호출 로그 확인
result2 = 3 | pipeline2 | to_str2
print(result2)

[CALL] double2(3)
[CALL] increment2(6)
[CALL] to_str2(7)
결과: 7
